In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from sklearn import model_selection, preprocessing, metrics
from sklearn.utils import shuffle
import xgboost as xgb
import os
import re
import time
#import h5py
import pickle
from tqdm import tqdm
#import cv2
from keras.models import Sequential, load_model
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda
from keras.layers import ELU
from keras.optimizers import Adam
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
color = sns.color_palette()

%matplotlib inline

In [ ]:
A = np.array([1, 3]).reshape((2, 1))
B = np.array([1, 2]).reshape((1, 2))
print(A.shape, B.shape)
np.dot(A, B)

In [ ]:
DATA_PATH = '/kaggle/dev/carvana-image-masking-challenge-data'
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw_data')
TRAIN_PATH = os.path.join(RAW_DATA_PATH, 'train')
TEST_PATH = os.path.join(RAW_DATA_PATH, 'test')
TRAIN_MASKS_PATH = os.path.join(RAW_DATA_PATH, 'train_masks')
TRAIN_MASKS_CSV_PATH = os.path.join(RAW_DATA_PATH, 'train_masks.csv')
SAMPLE_SUBMISSION_PATH = os.path.join(RAW_DATA_PATH, 'sample_submission.csv')
METADATA_PATH = os.path.join(RAW_DATA_PATH, 'metadata.csv')

In [ ]:
train_masks_df = pd.read_csv(TRAIN_MASKS_CSV_PATH)
print('train_masks_df.shape', train_masks_df.shape)
train_masks_df.head()

In [ ]:
metadata_df = pd.read_csv(METADATA_PATH)
print('metadata_df.shape', metadata_df.shape)
metadata_df.head()

In [ ]:
# Verify there are 5088 train images
num_train_images = len(os.listdir(TRAIN_PATH))
print('Training set contains {} cars with {} images each'.format(int(num_train_images/16), 16))

In [ ]:
# Verify there are 100064 test images
num_test_images = len(os.listdir(TEST_PATH))
print('Test set contains {} cars with {} images each'.format(int(num_test_images/16), 16))

In [ ]:
# Helper functions to plot car, mask, masked_car
def plot_image(img_id):
    try:
        img = mpimg.imread(os.path.join(TRAIN_PATH, img_id + ".jpg"))
    except OSError as e:
        img = mpimg.imread(os.path.join(TEST_PATH, img_id + ".jpg"))
    imgplot = plt.imshow(img)
    plt.axis('off')
    plt.show()
    
def plot_mask(img_id):
    try:
        mask = mpimg.imread(os.path.join(TRAIN_MASKS_PATH, img_id + "_mask.gif"))
    except OSError as e:
        mask = mpimg.imread(os.path.join(TEST_MASKS_PATH, img_id + "_mask.gif"))
    imgplot = plt.imshow(mask)
    plt.axis('off')
    plt.show()
    
def plot_masked_image(img_id):
    try:
        img = mpimg.imread(os.path.join(TRAIN_PATH, img_id + ".jpg"))
        mask = mpimg.imread(os.path.join(TRAIN_MASKS_PATH, img_id + "_mask.gif"))
    except OSError as e:
        img = mpimg.imread(os.path.join(TEST_PATH, img_id + ".jpg"))
        mask = mpimg.imread(os.path.join(TEST_MASKS_PATH, img_id + "_mask.gif"))
    mask = mask[:,:,0:3]
    mask_mul = mask.copy()
    mask_mul[mask == 255] = 1 
    masked_img = img * mask_mul
    imgplot = plt.imshow(masked_img)
    plt.axis('off')
    plt.show()

In [ ]:
# Plot random car, mask, masked_car
random_car_idx = int(np.random.random() * len(train_masks_df))
random_img_path = train_masks_df.iloc[random_car_idx].values[0]
m = re.match(r'([a-f0-9].*).jpg', random_img_path)
random_img_id = m.group(1)
print('Car id {}'.format(random_img_id))
plot_image(random_img_id)
plot_mask(random_img_id)
plot_masked_image(random_img_id)

In [ ]:
# Sample submission
sample_submission_df = pd.read_csv(SAMPLE_SUBMISSION_PATH)
print('sample_submission_df.shape', sample_submission_df.shape)
sample_submission_df.head()

In [ ]:
from keras.models import Model, load_model
from keras.layers import *
from keras.optimizers import Adam, RMSprop
from keras.losses import binary_crossentropy
import keras.backend.tensorflow_backend as KTF
def simple_model():
    inputs = Input(shape=(2,2,2))
    x = UpSampling2D((1,2))(inputs)
    #x = MaxPooling2D((2, 2), strides=(2, 2))(x)
    model = Model(inputs=inputs, outputs=x)
    return model

model = simple_model()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.1))
model.summary()

In [ ]:
random_input = np.asarray([[[[1, 2], [3, 4]], [[5, 6], [7, 8]]]])
print(random_input[0])
#print(random_input.shape)
model.predict(random_input)[0]